# Assignment 6

Maksim Nikiforov

## Split the data (do once)

Using the NFL box score data set, split the data into separate .csv files based on the season.

In [2]:
import pandas as pd

# Read in our .csv file
NFL_scores = pd.read_csv("scoresFull.csv")

# Show the first set of rows
NFL_scores.head()

,week,date,day,season,awayTeam,AQ1,AQ2,AQ3,AQ4,AOT,...,homeFumLost,homeNumPen,homePenYds,home3rdConv,home3rdAtt,home4thConv,home4thAtt,homeTOP,HminusAScore,homeSpread
0,1,5-Sep,Thu,2002,San Francisco 49ers,3,0,7,6,-1,...,0,10,80,4,8,0,1,32.47,-3,-4.0
1,1,8-Sep,Sun,2002,Minnesota Vikings,3,17,0,3,-1,...,1,4,33,2,6,0,0,28.48,4,4.5
2,1,8-Sep,Sun,2002,New Orleans Saints,6,7,7,0,6,...,0,8,85,1,6,0,1,31.48,-6,6.0
3,1,8-Sep,Sun,2002,New York Jets,0,17,3,11,6,...,1,10,82,4,8,2,2,39.13,-6,-3.0
4,1,8-Sep,Sun,2002,Arizona Cardinals,10,3,3,7,-1,...,0,7,56,6,10,1,2,34.40,8,6.0


In [3]:
# Group data frame by each season with groupby,
# then write to .csv using .to_csv() as covered in module 3.
# This produces a set of .csv files in the local directory that 
# begin with "season_20xx.csv".
for (season), group in NFL_scores.groupby(['season']):
     group.to_csv(f'season_{season}.csv', index=False)

## MapReduce

Consider a variable to group on (like _week_) and a numeric target variable (like _AQ1_).  Write a map function to find the

- sum of the target variable across the grouping variable
- sum of the squared values of the target variable across the grouping variable
- count or number of observations of the target variable in each group

In [47]:
# Read in and store each output data set as an element of the list season_data_list, which is iterable.
# Start with an empty list.
season_data_list = []

# Our .csv files end in a year (e.g. 2002), so iterate through all available years.
for year in range(2002, 2015):
    # Read in each season_20xx.csv file and append it to a list
    df = pd.read_csv('season_' + str(year) + '.csv')
    season_data_list.append(df)

# Verify that the number of elements in our season_data_list matches the
# number of .csv files we produced in the previous section.
len(season_data_list)

13

In [75]:
# Write the map function. 
# Take in one of the data sets (one season of data), a grouping variable (e.g. week), and a target variable (e.g. AQ1).
def map_stats(data, groupvar, targetvar):
    # Square the values of the target variable and store the result in a new column
    data[str(targetvar) + "_squared"] = data[targetvar].pow(2)
    # Subset the data frame to only include the grouping column, the target column, 
    # and the new column with squared values
    squared_targetvar = data[[groupvar, targetvar, str(targetvar) + "_squared"]]
    # Group the data using the grouping variable and display the sum and count for each column.
    # reset_index() to ensure that "week" ends up as a column
    grouped_data = squared_targetvar.groupby(groupvar).agg({targetvar: ["sum", "count"], str(targetvar) + "_squared": ["sum"]}).reset_index()
    # Convert to a dictionary of key-value pairs and return the result, referencing instructions from
    # https://stackoverflow.com/questions/26716616/convert-a-pandas-dataframe-to-a-dictionary
    grouped_data_list = grouped_data.set_index(groupvar).T.to_dict("list")
    # Lists are ordered as [sum_of_targetvar, count, sum_of_squared_targetvar]
    return grouped_data_list

In [76]:
# Confirm that the mapping function works for one season 
# using data from season_2002.csv.
# Recall that list elements are ordered as [sum_of_targetvar, count, sum_of_squared_targetvar]
map_stats(sample_data, "week", "AQ1")

{'1': [81, 16, 695],
 '10': [78, 14, 686],
 '11': [52, 16, 450],
 '12': [68, 16, 666],
 '13': [56, 16, 534],
 '14': [88, 16, 908],
 '15': [50, 16, 388],
 '16': [92, 16, 906],
 '17': [51, 16, 419],
 '2': [67, 16, 695],
 '3': [44, 14, 314],
 '4': [69, 14, 459],
 '5': [30, 14, 272],
 '6': [70, 14, 708],
 '7': [23, 14, 125],
 '8': [34, 14, 312],
 '9': [71, 14, 661],
 'ConfChamp': [17, 2, 149],
 'Division': [6, 4, 18],
 'SuperBowl': [3, 1, 9],
 'WildCard': [28, 4, 294]}

In [69]:
# Map our custom function to the entire list of seasons using the map() function, which
# accepts a function and applies it to each element of an iterable.
# Turn the result into a list, rather than keeping it as a basic map object
mapped_scores = list(map(map_stats, season_data_list, ["week"]*len(season_data_list), ["AQ1"]*len(season_data_list))) 

In [70]:
# look at just the 2003 season (mapped[1]) to confirm that our
# map() function worked as expected.
print("2003 season results: \nweek: [sum of AQ1, number of observations, sum of squared values of AQ1]")
for key, value in mapped_scores[1].items(): 
    print(key, ":", value)

2003 season results: 
week: [sum of AQ1, number of observations, sum of squared values of AQ1]
1 : [42, 16, 252]
10 : [51, 14, 419]
11 : [37, 16, 265]
12 : [40, 16, 446]
13 : [58, 16, 412]
14 : [53, 16, 397]
15 : [43, 16, 365]
16 : [78, 16, 808]
17 : [37, 16, 405]
2 : [85, 16, 913]
3 : [36, 14, 218]
4 : [68, 14, 750]
5 : [43, 14, 319]
6 : [39, 14, 243]
7 : [81, 14, 793]
8 : [65, 14, 505]
9 : [46, 14, 328]
ConfChamp : [0, 2, 0]
Division : [35, 4, 441]
SuperBowl : [0, 1, 0]
WildCard : [13, 4, 67]


Write a reduce function that takes in two dictionaries and combines them. 

In [64]:
# Reduce function that takes in two dictionaries and combines them.
# Check for unique keys (dictionary keys must be unique).
# If keys overlap, combine their list elements (values) into a new dictionary ("combined"). 
# Otherwise, create a new key.
def season_reduce(dict1, dict2):
    combined = {}
    for key in dict1.keys():
        if key in dict2:
            # List comprehension to combine values from two lists
            # into one list
            combined[key] = [x + y for x, y in zip(dict1[key], dict2[key])]
        else:
            combined[key] = dict1[key]
    for key in dict2.keys():
        if key not in dict1.keys():
            combined[key] = dict2[key]
    return combined

Check that the function works.

In [65]:
# Combine the 2002 and 2003 seasons using our reduce function.
season_02 = pd.read_csv("season_2002.csv")
first_season = map_stats(season_02, "week", "AQ1")

season_03 = pd.read_csv("season_2003.csv")
second_season = map_stats(season_03, "week", "AQ1")

temp = season_reduce(first_season, second_season)
for key, value in temp.items():
    print(key, ":", value)

1 : [123, 32, 947]
10 : [129, 28, 1105]
11 : [89, 32, 715]
12 : [108, 32, 1112]
13 : [114, 32, 946]
14 : [141, 32, 1305]
15 : [93, 32, 753]
16 : [170, 32, 1714]
17 : [88, 32, 824]
2 : [152, 32, 1608]
3 : [80, 28, 532]
4 : [137, 28, 1209]
5 : [73, 28, 591]
6 : [109, 28, 951]
7 : [104, 28, 918]
8 : [99, 28, 817]
9 : [117, 28, 989]
ConfChamp : [17, 4, 149]
Division : [41, 8, 459]
SuperBowl : [3, 2, 9]
WildCard : [41, 8, 361]


Use `functools.reduce()`. 

In [17]:
import functools

# season_reduce() is a function with 2 arguments
# season_data_list is a list (an iterable) of dictionaries
final = functools.reduce(season_reduce, mapped_scores)
for key, val in list(final.items()):
    print(key, ":", val)

1 : [713, 208, 5825]
10 : [852, 186, 7568]
11 : [798, 201, 7242]
12 : [749, 205, 6529]
13 : [872, 208, 8356]
14 : [798, 208, 7258]
15 : [814, 208, 7628]
16 : [852, 208, 7210]
17 : [775, 208, 7171]
2 : [731, 207, 6207]
3 : [832, 198, 7428]
4 : [730, 184, 6254]
5 : [649, 182, 5869]
6 : [750, 181, 6966]
7 : [776, 180, 7356]
8 : [595, 178, 4715]
9 : [744, 178, 7930]
ConfChamp : [83, 26, 757]
Division : [243, 52, 2529]
SuperBowl : [38, 13, 226]
WildCard : [229, 52, 2421]


## Last step

Write a function that takes the final result and returns the mean and standard deviation of the target variable at each level of the grouping variable. Recall that the list elements returned by our functions are ordered as `[sum_of_targetvar, count, sum_of_squared_targetvar]`.

In [81]:
# Accept a dictionary of the format 
# {week: [sum_of_targetvar, count, sum_of_squared_targetvar], ...},
# e.g. {'1': [713, 208, 5825],...}
def find_mean_std(weekly_stats_dict):
    means_dict = {}
    for key, val in list(weekly_stats_dict.items()):
        # Find the mean
        mean_val = round(val[0]/val[1], 2)
        # Check that "count" is greater than 1
        if val[1] > 1:
            # Find the sample standard deviation
            standard_dev = round(((val[2]-val[1]*(mean_val**2))/(val[1]-1))**0.5, 2)
        # Don’t return a standard deviation if 
        # the count is not greater than 1
        else:
            standard_dev = "NaN"
        # Create and return a dictionary of {week: [mean_val, sample_standard_dev], ...}
        means_dict[key] = [mean_val, standard_dev]
    return means_dict

# Test the function against our complete data set
find_mean_std(final)

{'1': [3.43, 4.04],
 '10': [4.58, 4.45],
 '11': [3.97, 4.51],
 '12': [3.65, 4.31],
 '13': [4.19, 4.77],
 '14': [3.84, 4.5],
 '15': [3.91, 4.64],
 '16': [4.1, 4.24],
 '17': [3.73, 4.55],
 '2': [3.53, 4.2],
 '3': [4.2, 4.47],
 '4': [3.97, 4.28],
 '5': [3.57, 4.43],
 '6': [4.14, 4.63],
 '7': [4.31, 4.73],
 '8': [3.34, 3.93],
 '9': [4.18, 5.22],
 'ConfChamp': [3.19, 4.44],
 'Division': [4.67, 5.23],
 'SuperBowl': [2.92, 3.1],
 'WildCard': [4.4, 5.27]}

Write a function that takes in the data (always the same iterable of our data sets), a target variable, and a grouping variable. The function should call our functions above to easily produce the final result with a single function call!

In [71]:
def find_NFL_stats(data, groupvar, targetvar):
    mapped_scores = list(map(map_stats, data, [groupvar]*len(data), [targetvar]*len(data))) 
    final = functools.reduce(season_reduce, mapped_scores)
    return find_mean_std(final)

In [82]:
find_NFL_stats(season_data_list, "day", "AQ2")

{'Mon': [6.09, 5.55],
 'Sat': [6.92, 5.56],
 'Sun': [6.2, 5.13],
 'Thu': [6.89, 6.29],
 'Fri': [12.67, 5.12],
 'Tue': [7.0, 'NaN'],
 'Wed': [7.0, 'NaN']}